In [3]:
#IMPORTS
import pandas as pd
import tensorflow as tf


In [13]:
#GET DATA
!wget https://raw.githubusercontent.com/bbr32/nlp_proj/master/dataset_test_no_labels.csv
!wget https://raw.githubusercontent.com/bbr32/nlp_proj/master/dataset_train.csv

--2019-12-17 16:53:51--  https://raw.githubusercontent.com/bbr32/nlp_proj/master/dataset_test_no_labels.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3537628 (3.4M) [text/plain]
Saving to: ‘dataset_test_no_labels.csv.1’

dataset_test_no_lab 100%[===================>]   3.37M  --.-KB/s    in 0.06s   

2019-12-17 16:53:52 (56.9 MB/s) - ‘dataset_test_no_labels.csv.1’ saved [3537628/3537628]

--2019-12-17 16:53:53--  https://raw.githubusercontent.com/bbr32/nlp_proj/master/dataset_train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7

In [16]:
df_train = pd.read_csv('dataset_train.csv', sep='\t')
df_train.head()

,index,sentence_1,sentence_2,label
0,0,Conceptually cream skimming has two basic dime...,Product and geography are what make cream skim...,neutral
1,1,you know during the season and i guess at at y...,You lose the things to the following level if ...,entailment
2,2,One of our number will carry out your instruct...,A member of my team will execute your orders w...,entailment
3,3,How do you know? All this is their information...,This information belongs to them.,entailment
4,4,yeah i tell you what though if you go price so...,The tennis shoes have a range of prices.,neutral


In [17]:
df_test = pd.read_csv('dataset_test_no_labels.csv', sep='\t')
df_test.head()

,index,sentence_1,sentence_2
0,0,One senior FAA air traffic control manager sai...,A senior FFA air traffic control manager said ...
1,1,"Once constructed, this persisting psychologica...",Once unique experiences become personally mean...
2,2,Help us continue to offer the finest professio...,Support us to keep offering the best professio...
3,3,"At the end of the Wars of Spanish, Austrian, a...",Northern Italy was not easily given up to the ...
4,4,Update on the Democratic fund-raising scandal ...,Clinton said the agents had not told him anyth...


In [0]:
#SENTENCE TO SEQUENCE

In [0]:
#VISUALIZE